# 

In [1]:
!pip install --quiet jupyterlab-vim jupytex
!jupyter labextension enable

%load_ext autoreload
%autoreload 2

In [3]:
#!pip install quiet langchain==0.1.0 openai==1.7.2 langchain-openai==0.0.2 langchain-community==0.0.12 langchainhub==0.1.14
!pip install --quiet langchain openai langchain-openai langchain-community langchainhub
!pip install --quiet python-dotenv

In [3]:
# https://github.com/hfhoffman1144/langchain_neo4j_rag_app/tree/main
# git clone git@github.com:hfhoffman1144/langchain_neo4j_rag_app.git
# /Users/saggese/src/github/langchain_neo4j_rag_app

# Step 1, Intro to LangChain

In [4]:
import os; os.environ["OPENAI_API_KEY"] = ""

In [9]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [10]:
from langchain.schema.messages import HumanMessage, SystemMessage
#from langchain_intro.chatbot import chat_model

messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about
        healthcare. Only answer healthcare-related questions."""
    ),
    HumanMessage(content="What is Medicaid managed care?"),
]
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which states contract with managed care organizations (MCOs) to provide healthcare services to Medicaid beneficiaries. These MCOs are responsible for coordinating and delivering healthcare services to enrollees in exchange for a fixed monthly payment per enrollee. Medicaid managed care aims to improve access to care, enhance quality of services, and control costs for the Medicaid program.')

In [11]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about
        healthcare. Only answer healthcare-related questions."""
    ),
    HumanMessage(content="How do I change a tire?"),
]
chat_model.invoke(messages)

AIMessage(content="I'm here to help with healthcare-related questions. If you have any health-related inquiries, feel free to ask!")

In [13]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = ChatPromptTemplate.from_template(review_template_str)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

print(review_template.format(context=context, question=question))

Human: Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

I had a great stay!

Did anyone have a positive experience?



In [19]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

ret = review_prompt_template.format_messages(context=context, question=question)
print(ret)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"), HumanMessage(content='Did anyone have a positive experience?')]


## Chains

In [23]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at
a hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=review_template_str,
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="{question}",
    )
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

output_parser = StrOutputParser()

review_chain = review_prompt_template | chat_model | output_parser

In [24]:
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_chain.invoke({"context": context, "question": question})

'Yes, the patient had a great stay at the hospital, indicating a positive experience.'

## Retrieval

In [26]:
!pip install --quiet chromadb

In [34]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

#!cp /Users/saggese/src/github/langchain_neo4j_rag_app/data/reviews.csv build_LLM_RAG_chatbot_with_langchain/
REVIEWS_CSV_PATH = "build_LLM_RAG_chatbot_with_langchain/reviews.csv"
REVIEWS_CHROMA_PATH = "chroma_data"

loader = CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="review")
reviews = loader.load()
#!head $REVIEWS_CSV_PATH

reviews_vector_db = Chroma.from_documents(
    reviews, OpenAIEmbeddings(), persist_directory=REVIEWS_CHROMA_PATH
)

In [40]:
reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=OpenAIEmbeddings(),
)

#question = """Has anyone complained about
#           communication with the hospital staff?"""
question = """physician_name Maria Thompson"""
relevant_docs = reviews_vector_db.similarity_search(question, k=3)

print(relevant_docs[0].page_content)
print(relevant_docs[1].page_content)

review_id: 154
visit_id: 6491
review: The level of care I received from the nurses exceeded my expectations. However, the billing process afterward was confusing, and it took multiple calls to resolve the issues.
physician_name: Tamara Potter
hospital_name: Malone, Thompson and Mejia
patient_name: Mark Morgan
review_id: 154
visit_id: 6491
review: The level of care I received from the nurses exceeded my expectations. However, the billing process afterward was confusing, and it took multiple calls to resolve the issues.
physician_name: Tamara Potter
hospital_name: Malone, Thompson and Mejia
patient_name: Mark Morgan


In [42]:
# Pass the relevant reviews to the prompt as content.

from langchain.schema.runnable import RunnablePassthrough

# Find 10 reviews closer.
reviews_retriever  = reviews_vector_db.as_retriever(k=10)

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)

In [43]:
question = """Has anyone complained about
           communication with the hospital staff?"""
review_chain.invoke(question)

'Yes, a patient named Terri Smith complained about the communication between the medical staff and herself at the hospital. She mentioned that the communication was unclear, leading to misunderstandings about her treatment plan. Terri Smith suggested that improvement is needed in this area.'

## Agents

- The chain is hardwired
- An agent is an LLM that decides the sequence of actions to execute.

In [44]:
import random
import time

def get_current_wait_time(hospital: str) -> int | str:
    """Dummy function to generate fake wait times"""

    if hospital not in ["A", "B", "C", "D"]:
        return f"Hospital {hospital} does not exist"

    # Simulate API call delay
    time.sleep(1)

    return random.randint(0, 10000)

In [45]:
from langchain.agents import (
    create_openai_functions_agent,
    Tool,
    AgentExecutor,
)
from langchain import hub

# Tool is an interface that an agent uses to interact with a function.
# Each description explains the Agent when to call each tool.
tools = [
    Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions
        about patient reviews or experiences at the hospital.
        Not useful for answering questions about specific visit
        details such as payer, billing, treatment, diagnosis,
        chief complaint, hospital, or physician information.
        Pass the entire question as input to the tool. For instance,
        if the question is "What do patients think about the triage system?",
        the input should be "What do patients think about the triage system?"
        """,
    ),
    Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]

hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")

agent_chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    temperature=0,
)

hospital_agent = create_openai_functions_agent(
    llm=agent_chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)

# Agent run-time.
hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

In [46]:
hospital_agent_executor.invoke(
    {"input": "What is the current wait time at hospital C?"}
)



> Entering new AgentExecutor chain...

Invoking: `Waits` with `C`


833The current wait time at hospital C is 833 minutes.

> Finished chain.


{'input': 'What is the current wait time at hospital C?',
 'output': 'The current wait time at hospital C is 833 minutes.',
 'intermediate_steps': [(AgentActionMessageLog(tool='Waits', tool_input='C', log='\nInvoking: `Waits` with `C`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"C"}', 'name': 'Waits'}})]),
   833)]}

In [47]:
hospital_agent_executor.invoke(
    {"input": "What have patients said about their comfort at the hospital?"}
)



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `What have patients said about their comfort at the hospital?`


Patients have mentioned that the hospital staff is dedicated to patient care, but they have also expressed discomfort due to the uncomfortable beds, making it difficult to get a good night's sleep during their stay.Patients have mentioned that the hospital staff is dedicated to patient care, but they have also expressed discomfort due to the uncomfortable beds, making it difficult to get a good night's sleep during their stay.

> Finished chain.


{'input': 'What have patients said about their comfort at the hospital?',
 'output': "Patients have mentioned that the hospital staff is dedicated to patient care, but they have also expressed discomfort due to the uncomfortable beds, making it difficult to get a good night's sleep during their stay.",
 'intermediate_steps': [(AgentActionMessageLog(tool='Reviews', tool_input='What have patients said about their comfort at the hospital?', log='\nInvoking: `Reviews` with `What have patients said about their comfort at the hospital?`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"What have patients said about their comfort at the hospital?"}', 'name': 'Reviews'}})]),
   "Patients have mentioned that the hospital staff is dedicated to patient care, but they have also expressed discomfort due to the uncomfortable beds, making it difficult to get a good night's sleep during their stay.")]}

# Step 2, Understand the business requirements and data

In [51]:
import pandas as pd

data = pd.read_csv("build_LLM_RAG_chatbot_with_langchain/hospitals.csv")
print(data.shape)
data.head()

(30, 3)


,hospital_id,hospital_name,hospital_state
0,0,Wallace-Hamilton,CO
1,1,"Burke, Griffin and Cooper",NC
2,2,Walton LLC,FL
3,3,Garcia Ltd,NC
4,4,"Jones, Brown and Murray",NC


In [52]:
data = pd.read_csv("build_LLM_RAG_chatbot_with_langchain/physicians.csv")
print(data.shape)
data.head()

(500, 6)


,physician_name,physician_id,physician_dob,physician_grad_year,medical_school,salary
0,Joseph Johnson,0,1970-02-22,2000-02-22,Johns Hopkins University School of Medicine,309534.155076
1,Jason Williams,1,1982-12-22,2012-12-22,Mayo Clinic Alix School of Medicine,281114.503559
2,Jesse Gordon,2,1959-06-03,1989-06-03,David Geffen School of Medicine at UCLA,305845.584636
3,Heather Smith,3,1965-06-15,1995-06-15,NYU Grossman Medical School,295239.766689
4,Kayla Hunter DDS,4,1978-10-19,2008-10-19,David Geffen School of Medicine at UCLA,298751.355201


In [53]:
data = pd.read_csv("build_LLM_RAG_chatbot_with_langchain/payers.csv")
print(data.shape)
data.head()

(5, 2)


,payer_name,payer_id
0,Medicaid,0
1,UnitedHealthcare,1
2,Aetna,2
3,Cigna,3
4,Blue Cross,4


In [54]:
data = pd.read_csv("build_LLM_RAG_chatbot_with_langchain/reviews.csv")
print(data.shape)
data.head()

(1005, 6)


,review_id,visit_id,review,physician_name,hospital_name,patient_name
0,0,6997,The medical staff at the hospital were incredi...,Laura Brown,Wallace-Hamilton,Christy Johnson
1,9,8138,The hospital's commitment to patient education...,Steven Watson,Wallace-Hamilton,Anna Frazier
2,11,680,The hospital's commitment to patient safety wa...,Chase Mcpherson Jr.,Wallace-Hamilton,Abigail Mitchell
3,892,9846,I had a positive experience overall at the hos...,Jason Martinez,Wallace-Hamilton,Kimberly Rivas
4,822,7397,The medical team at the hospital was exception...,Chelsey Davis,Wallace-Hamilton,Catherine Yang


In [ ]:
data = pd.read_csv("visits/reviews.csv")
print(data.shape)
data.head()

In [55]:
data = pd.read_csv("build_LLM_RAG_chatbot_with_langchain/visits.csv")
print(data.shape)
data.head()

(9998, 15)


,patient_id,date_of_admission,billing_amount,room_number,admission_type,discharge_date,test_results,visit_id,physician_id,payer_id,hospital_id,chief_complaint,treatment_description,primary_diagnosis,visit_status
0,0,2022-11-17,37490.983364,146,Elective,2022-12-01,Inconclusive,0,102,1,0,NaN,NaN,NaN,DISCHARGED
1,1,2023-06-01,47304.064845,404,Emergency,NaN,Normal,1,435,4,5,NaN,NaN,NaN,OPEN
2,2,2019-01-09,36874.896997,292,Emergency,2019-02-08,Normal,2,348,2,6,NaN,NaN,NaN,DISCHARGED
3,3,2020-05-02,23303.322092,480,Urgent,2020-05-03,Abnormal,3,270,4,15,NaN,NaN,NaN,DISCHARGED
4,4,2021-07-09,18086.344184,477,Urgent,2021-08-02,Normal,4,106,2,29,Persistent cough and shortness of breath,Prescribed a combination of inhaled bronchodil...,"J45.909 - Unspecified asthma, uncomplicated",DISCHARGED


# Step 3. Neo4j

In [ ]:
- Nodes (Patient, Visit, Payer)
- Relationships (Has, Covered_by)
- Properties (associated to Patient, Has, Covered_by, Payer)

## ETL

In [68]:
import os
import logging
#from retry import retry
from neo4j import GraphDatabase

root_dir = "https://raw.githubusercontent.com/hfhoffman1144/langchain_neo4j_rag_app/main/data/"

# Paths to CSV files containing hospital data
#HOSPITALS_CSV_PATH = "file:///build_LLM_RAG_chatbot_with_langchain/hospitals.csv"
HOSPITALS_CSV_PATH = root_dir + "hospitals.csv"
PAYERS_CSV_PATH = root_dir + "payers.csv"
PHYSICIANS_CSV_PATH = root_dir + "physicians.csv"
PATIENTS_CSV_PATH = root_dir + "patients.csv"
VISITS_CSV_PATH = root_dir + "visits.csv"
REVIEWS_CSV_PATH = root_dir + "reviews.csv"
EXAMPLE_CYPHER_CSV_PATH = root_dir + "example_cypher.csv"

# Neo4j config
NEO4J_URI = "bolt://neo4j:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "testtest"

NODES = ["Hospital", "Payer", "Physician", "Patient", "Visit", "Review", "Question"]

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

driver.verify_connectivity()

In [78]:
import hneo4j

hneo4j.delete_all(driver)

In [79]:
def _set_uniqueness_constraints(tx, node):
    query = f"""CREATE CONSTRAINT IF NOT EXISTS FOR (n:{node})
        REQUIRE n.id IS UNIQUE;"""
    _ = tx.run(query, {})


with driver.session(database="neo4j") as session:
    for node in NODES:
        session.execute_write(_set_uniqueness_constraints, node)

In [80]:
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{HOSPITALS_CSV_PATH}' AS hospitals
    MERGE (h:Hospital {{id: toInteger(hospitals.hospital_id),
                        name: hospitals.hospital_name,
                        state_name: hospitals.hospital_state}});
    """
    _ = session.run(query, {})

In [81]:
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{HOSPITALS_CSV_PATH}' AS hospitals
    MERGE (h:Hospital {{id: toInteger(hospitals.hospital_id),
                        name: hospitals.hospital_name,
                        state_name: hospitals.hospital_state}});
    """
    _ = session.run(query, {})

print("Loading payer nodes")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{PAYERS_CSV_PATH}' AS payers
    MERGE (p:Payer {{id: toInteger(payers.payer_id),
    name: payers.payer_name}});
    """
    _ = session.run(query, {})

print("Loading physician nodes")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{PHYSICIANS_CSV_PATH}' AS physicians
    MERGE (p:Physician {{id: toInteger(physicians.physician_id),
                        name: physicians.physician_name,
                        dob: physicians.physician_dob,
                        grad_year: physicians.physician_grad_year,
                        school: physicians.medical_school,
                        salary: toFloat(physicians.salary)
                        }});
    """
    _ = session.run(query, {})

print("Loading visit nodes")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS_CSV_PATH}' AS visits
    MERGE (v:Visit {{id: toInteger(visits.visit_id),
                        room_number: toInteger(visits.room_number),
                        admission_type: visits.admission_type,
                        admission_date: visits.date_of_admission,
                        test_results: visits.test_results,
                        status: visits.visit_status
    }})
        ON CREATE SET v.chief_complaint = visits.chief_complaint
        ON MATCH SET v.chief_complaint = visits.chief_complaint
        ON CREATE SET v.treatment_description =
        visits.treatment_description
        ON MATCH SET v.treatment_description = visits.treatment_description
        ON CREATE SET v.diagnosis = visits.primary_diagnosis
        ON MATCH SET v.diagnosis = visits.primary_diagnosis
        ON CREATE SET v.discharge_date = visits.discharge_date
        ON MATCH SET v.discharge_date = visits.discharge_date
     """
    _ = session.run(query, {})

print("Loading patient nodes")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{PATIENTS_CSV_PATH}' AS patients
    MERGE (p:Patient {{id: toInteger(patients.patient_id),
                    name: patients.patient_name,
                    sex: patients.patient_sex,
                    dob: patients.patient_dob,
                    blood_type: patients.patient_blood_type
                    }});
    """
    _ = session.run(query, {})

print("Loading review nodes")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{REVIEWS_CSV_PATH}' AS reviews
    MERGE (r:Review {{id: toInteger(reviews.review_id),
                     text: reviews.review,
                     patient_name: reviews.patient_name,
                     physician_name: reviews.physician_name,
                     hospital_name: reviews.hospital_name
                    }});
    """
    _ = session.run(query, {})

print("Loading question nodes")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{EXAMPLE_CYPHER_CSV_PATH}' AS questions
    MERGE (Q:Question {{
                     question: questions.question,
                     cypher: questions.cypher
                    }});
    """
    _ = session.run(query, {})

print("Loading 'AT' relationships")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS_CSV_PATH}' AS row
    MATCH (source: `Visit` {{ `id`: toInteger(trim(row.`visit_id`)) }})
    MATCH (target: `Hospital` {{ `id`:
    toInteger(trim(row.`hospital_id`))}})
    MERGE (source)-[r: `AT`]->(target)
    """
    _ = session.run(query, {})

print("Loading 'WRITES' relationships")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{REVIEWS_CSV_PATH}' AS reviews
        MATCH (v:Visit {{id: toInteger(reviews.visit_id)}})
        MATCH (r:Review {{id: toInteger(reviews.review_id)}})
        MERGE (v)-[writes:WRITES]->(r)
    """
    _ = session.run(query, {})

print("Loading 'TREATS' relationships")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS_CSV_PATH}' AS visits
        MATCH (p:Physician {{id: toInteger(visits.physician_id)}})
        MATCH (v:Visit {{id: toInteger(visits.visit_id)}})
        MERGE (p)-[treats:TREATS]->(v)
    """
    _ = session.run(query, {})

print("Loading 'COVERED_BY' relationships")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS_CSV_PATH}' AS visits
        MATCH (v:Visit {{id: toInteger(visits.visit_id)}})
        MATCH (p:Payer {{id: toInteger(visits.payer_id)}})
        MERGE (v)-[covered_by:COVERED_BY]->(p)
        ON CREATE SET
            covered_by.service_date = visits.discharge_date,
            covered_by.billing_amount = toFloat(visits.billing_amount)
    """
    _ = session.run(query, {})

print("Loading 'HAS' relationships")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS_CSV_PATH}' AS visits
        MATCH (p:Patient {{id: toInteger(visits.patient_id)}})
        MATCH (v:Visit {{id: toInteger(visits.visit_id)}})
        MERGE (p)-[has:HAS]->(v)
    """
    _ = session.run(query, {})

print("Loading 'EMPLOYS' relationships")
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS_CSV_PATH}' AS visits
        MATCH (h:Hospital {{id: toInteger(visits.hospital_id)}})
        MATCH (p:Physician {{id: toInteger(visits.physician_id)}})
        MERGE (h)-[employs:EMPLOYS]->(p)
    """
    _ = session.run(query, {})

Loading payer nodes
Loading physician nodes
Loading visit nodes
Loading patient nodes
Loading review nodes
Loading question nodes
Loading 'AT' relationships
Loading 'WRITES' relationships
Loading 'TREATS' relationships
Loading 'COVERED_BY' relationships
Loading 'HAS' relationships
Loading 'EMPLOYS' relationships


In [87]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

graph.refresh_schema()

print(graph.schema)

Node properties are the following:
Hospital {id: INTEGER, name: STRING, state_name: STRING},Payer {name: STRING, id: INTEGER},Physician {salary: FLOAT, dob: STRING, grad_year: STRING, name: STRING, id: INTEGER, school: STRING},Patient {id: INTEGER, dob: STRING, blood_type: STRING, sex: STRING, name: STRING},Visit {treatment_description: STRING, diagnosis: STRING, id: INTEGER, status: STRING, discharge_date: STRING, admission_type: STRING, test_results: STRING, admission_date: STRING, room_number: INTEGER, chief_complaint: STRING},Review {hospital_name: STRING, patient_name: STRING, physician_name: STRING, text: STRING, id: INTEGER},Question {cypher: STRING, question: STRING}
Relationship properties are the following:
COVERED_BY {service_date: STRING, billing_amount: FLOAT}
The relationships are the following:
(:Hospital)-[:EMPLOYS]->(:Physician),(:Physician)-[:TREATS]->(:Visit),(:Patient)-[:HAS]->(:Visit),(:Visit)-[:AT]->(:Hospital),(:Visit)-[:COVERED_BY]->(:Payer),(:Visit)-[:WRITES]->

In [92]:
type(schema)

str

In [90]:
from pyvis.network import Network

# Refresh the schema
graph.refresh_schema()

# Extract the schema
schema = graph.schema

# Create a Pyvis network graph
#net = Network(notebook=True)
net = Network(cdn_resources = "remote", directed = True, height = '500px',width = '100%',
          notebook = True)

# Add nodes and edges from the schema to the Pyvis graph
for label in schema['labels']:
    net.add_node(label, label=label)

for relationship in schema['relationships']:
    start_label = relationship['start_node_label']
    end_label = relationship['end_node_label']
    rel_type = relationship['type']
    net.add_edge(start_label, end_label, title=rel_type)

# Show the graph
net.show("neo4j_schema.html")

TypeError: string indices must be integers

In [82]:
with driver.session(database="neo4j") as session:
    net = hneo4j.plot_graph(session)
net.show("graph.html")

graph.html
